In [1]:
import numpy as np
import h5py
import string
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import time
import glob
from util import crawl_meta, PaperMeta, Keyword
%matplotlib inline

In [2]:
CRAWL_DATA = False
AFTER_DECISION = False
CRAWL_REVIEW = False

In [ ]:
# Get the meta data
if CRAWL_DATA:
    # Uncomment this if you want to crawl data from scratch
    meta_list = crawl_meta(
        meta_hdf5=None, 
        write_meta_name='data_{}.hdf5'.format(time.strftime("%Y%m%d%H%M%S")), 
        crawl_review=CRAWL_REVIEW)
else:
    # Uncomment this if you want to load the previously stored data file
    meta_hdf5s = glob.glob("*.hdf5")
    meta_hdf5 = sorted(meta_hdf5s)[-1] # the most recent data
    print("Loading {} ...".format(meta_hdf5))
    meta_list = crawl_meta(meta_hdf5=meta_hdf5)
num_withdrawn = len([m for m in meta_list if m.withdrawn or m.desk_reject])
print('Number of submissions: {} (withdrawn/desk reject submissions: {})'.format(
    len(meta_list), num_withdrawn))

In [ ]:
rating = []
rating_mean = []
num_rating = []
keywords = []
for m in meta_list:
    rating.extend(m.rating)
    keywords.extend(m.keyword)
    if not (m.withdrawn or m.desk_reject):
        num_rating.append(len(m.rating))
    if len(m.rating) > 0:
        rating_mean.append(m.average_rating)
print('Average rating: {:.4f}'.format(np.mean(rating)))

In [ ]:
# Plot histograms of ratings
from collections import Counter
from matplotlib import gridspec
import seaborn as sns; sns.set()
fig = plt.figure(figsize=(15, 6))
plt.suptitle('Histograms of ICLR 2020 Submission Ratings')

# Rating hist
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3]) 
plt.subplot(gs[0])
counterlist =  sorted(Counter(rating).most_common())
frequencies = [k[1] for k in counterlist]
bins = [k[0] for k in counterlist]
freq_series = pd.Series.from_array(frequencies)

x_labels = ['{:.2f}'.format(b) for b in bins]

ax1 = freq_series.plot(kind='bar', color='#990000', alpha=0.75, width=0.9)
ax1.set_xlabel('Rating hist')
ax1.set_ylabel('Number of papers')
ax1.set_xticklabels(x_labels)

rects = ax1.patches
labels = [int(frequencies[i]) for i in range(len(rects))]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax1.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
            ha='center', va='bottom', fontsize=9)

# Rating per paper hist
plt.subplot(gs[1])
counterlist =  sorted(Counter(rating_mean).most_common())
frequencies = [k[1] for k in counterlist]
bins = [k[0] for k in counterlist]
freq_series = pd.Series.from_array(frequencies)

x_labels = ['{:.2f}'.format(b) for b in bins]

ax2 = freq_series.plot(kind='bar', color='#990000', alpha=0.75, width=0.9)
ax2.set_xlabel('Rating per paper hist')
ax2.set_xticklabels(x_labels, fontsize=9)

rects = ax2.patches
labels = [int(frequencies[i]) for i in range(len(rects))]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width() / 2, height + 3, label,
            ha='center', va='bottom', fontsize=9)
plt.show()
fig.savefig('asset/rating.png')

In [ ]:
# Plot histograms of ratings
from collections import Counter
from matplotlib import gridspec
import seaborn as sns; sns.set()
fig = plt.figure(figsize=(15, 6))
plt.suptitle('Histograms of ICLR 2020 Submission Ratings')

# Rating hist
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 7]) 
plt.subplot(gs[0])
counterlist =  sorted(Counter(rating).most_common())
frequencies = [k[1] for k in counterlist]
frequencies = [float(f) / np.sum(frequencies) * 100 for f in frequencies]
frequencies = np.cumsum(frequencies[::-1])[::-1]
bins = [k[0] for k in counterlist]
freq_series = pd.Series.from_array(frequencies)

x_labels = ['{:.2f}'.format(b) for b in bins]

ax1 = freq_series.plot(kind='bar', color='#990000', alpha=0.75, width=0.9)
ax1.set_xlabel('Rating')
ax1.set_ylabel('Percentage (%)')
ax1.set_xticklabels(x_labels)

rects = ax1.patches
labels = ['{:.1f}'.format(frequencies[i]) for i in range(len(rects))]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax1.text(rect.get_x() + rect.get_width() / 2, height + 1.5, label,
            ha='center', va='bottom', fontsize=8)

# Rating per paper hist
plt.subplot(gs[1])
counterlist =  sorted(Counter(rating_mean).most_common())
frequencies = [k[1] for k in counterlist]
frequencies = [float(f) / np.sum(frequencies) * 100 for f in frequencies]
frequencies = np.cumsum(frequencies[::-1])[::-1]
bins = [k[0] for k in counterlist]
freq_series = pd.Series.from_array(frequencies)

x_labels = ['{:.2f}'.format(b) for b in bins]

ax2 = freq_series.plot(kind='bar', color='#990000', alpha=0.75, width=0.9)
ax2.set_xlabel('Average Rating')
ax2.set_xticklabels(x_labels, fontsize=9)

rects = ax2.patches
labels = ['{:.1f}'.format(frequencies[i]) for i in range(len(rects))]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width() / 2, height + 1.5, label,
            ha='center', va='bottom', fontsize=5)
plt.show()
fig.savefig('asset/rating_cumsum.png')

In [ ]:
# See how many papers are beaten by yours
def PR(rating_mean, your_rating):
    pr = np.sum(your_rating > np.array(rating_mean))/len(rating_mean)*100
    same_rating = np.sum(your_rating == np.array(rating_mean))/len(rating_mean)*100    
    return pr, same_rating
my_rating = (6+6+3)/3.  # your average rating here
pr, same_rating = PR(rating_mean, my_rating)
print('Your papar ({:.2f}) is among the top {:.2f}% of submissions based on the ratings.\n'
      'There are {:.2f}% with the same rating.'.format(
          my_rating, 100-pr, same_rating))

#            accept rate       orals     posters
# ICLR 2017: 39.1% (198/507)    15         183
# ICLR 2018: 32.0% (314/981)    23         291
# ICLR 2019: 31.4% (500/1591)   24         476
# ICLR 2020: ?     (?/2594)

In [ ]:
# Count keywords
keywords_hist = Counter(keywords)
del keywords_hist['']
print('{} different keywords before merging'.format(len(keywords_hist)))

# Merge duplicates: CNNs and CNN
duplicates = []
for k in keywords_hist:
    if k+'s' in keywords_hist:
        duplicates.append(k)
for k in duplicates:
    keywords_hist[k] += keywords_hist[k+'s']
    del keywords_hist[k+'s']
print('{} different keywords after merging'.format(len(keywords_hist)))

In [ ]:
# Create keywords list
keywords_list = []
keywords_key_list = []
for m in meta_list:
    for k in [mk for mk in m.keyword if not mk == '']:
        if k not in keywords_hist.keys():
            k = k[:-1]  # strip 's'
        if k in keywords_key_list:
            idx = keywords_key_list.index(k)
            keywords_list[idx].update_frequency(1)
            keywords_list[idx].update_rating(m.rating)
        else:
            # the keyword is new to the list
            k_object = Keyword(k, 1, m.rating)
            keywords_list.append(k_object)
            keywords_key_list.append(k_object.keyword)

In [ ]:
keywords_list_subset = [k for k in keywords_list if k.frequency > 9]
print(len(keywords_list_subset))
y = [k.average_rating() for k in keywords_list_subset]
x = [np.log2(k.frequency) for k in keywords_list_subset]
key = [k.keyword for k in keywords_list_subset]

# Create dataframe
df = pd.DataFrame({
    'Frequency': x,
    'AverageRating': y,
    'Keyword': key
})

sns.set(font_scale=1)
fig = plt.figure(figsize=(12, 12))
p1 = sns.regplot(data=df, x="Frequency", y="AverageRating", fit_reg=False, 
                 marker="o", color="red", logx=True, scatter_kws={'s': 8})
for line in range(0, df.shape[0]):
     p1.text(df.Frequency[line], df.AverageRating[line], df.Keyword[line], 
             horizontalalignment='left', 
             size='small', color='black', alpha=0.6)
plt.show()
fig.savefig('asset/rating_frequency.png')

In [ ]:
# Show N most common keywords and their frequencies
num_keyowrd = 50
keywords_hist_vis = keywords_hist.most_common(num_keyowrd)

plt.rcdefaults()
fig, ax = plt.subplots(figsize=(8, 12))

key = [k[0] for k in keywords_hist_vis] 
value = [k[1] for k in keywords_hist_vis] 
y_pos = np.arange(len(key))
ax.barh(y_pos, value, align='center', color='green', ecolor='black', log=True)
ax.set_yticks(y_pos)
ax.set_yticklabels(key, rotation=0, fontsize=10)
ax.invert_yaxis() 
for i, v in enumerate(value):
    ax.text(v + 3, i + .25, str(v), color='black', fontsize=10)
# ax.text(y_pos, value, str(value))
ax.set_xlabel('Frequency')
ax.set_title('ICLR 2020 Submission Top {} Keywords'.format(num_keyowrd))

plt.show()
fig.savefig('asset/frequency.png')

In [ ]:
# Show the word cloud forming by keywords
from wordcloud import WordCloud
wordcloud = WordCloud(max_font_size=64, max_words=160, 
                      width=1280, height=640,
                      background_color="black").generate(' '.join(keywords))
fig = plt.figure(figsize=(16, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
fig.savefig('asset/wordcloud.png')

In [ ]:
# Show the word cloud with an ICLR logo
from imageio import imread
logo = imread('asset/logo.png')
wordcloud = WordCloud(max_font_size=64, max_words=300, 
                      width=1280, height=640,
                      background_color="white", mask=logo).generate(' '.join(keywords))
fig = plt.figure(figsize=(16, 8), frameon=False)
plt.imshow(logo)
plt.imshow(wordcloud, interpolation="bilinear",  alpha=.7)
plt.axis("off")
plt.show()
fig.savefig('asset/logo_wordcloud.png')

In [ ]:
num_valid = len(meta_list) - num_withdrawn
num_missing_rating = num_valid*3 - np.sum(np.clip(num_rating, 0, 3))
print('Number of missing reviews: {} ({:.4f}%)'.format(
    num_missing_rating, 100*float(num_missing_rating)/(num_valid*3)))

In [ ]:
# the papers that don't have 3 reviews
count = 0
for m in meta_list:
    if len(m.rating) < 3 and not (m.withdrawn or m.desk_reject):
        count += 1
        print('[{}] {} {} {}'.format(count, m.title, m.url, m.rating))

In [ ]:
import operator
meta_list_sorted_review_len_min = sorted(
    [m for m in meta_list if not m.review_len_min is None], 
    key=operator.attrgetter('review_len_min'))
meta_list_sorted_review_len_max = sorted(
    [m for m in meta_list if not m.review_len_max is None], 
    key=operator.attrgetter('review_len_max'), reverse=True)

In [ ]:
TOP_NUM = 10
print('[Shortest reviews]')
for m in meta_list_sorted_review_len_min[:TOP_NUM]:
    print('{} words: {} {}'.format(m.review_len_min, m.url, m.title))
print('[Longest reviews]')    
for m in meta_list_sorted_review_len_max[:TOP_NUM]:
    print('{} words: {} {}'.format(m.review_len_max, m.url, m.title))        

In [ ]:
# average review len
all_rating = []
all_review_len = []
for m in meta_list:
    if len(m.review_len) > 0:
        all_rating.extend(m.rating)
        all_review_len.extend(m.review_len)        
print('The average review length: {:.2f} words'.format(np.mean(all_review_len)))

In [ ]:
# Plot histograms of ratings
sns.set()
fig = plt.figure(figsize=(12, 6))
plt.title('Histogram of ICLR 2020 Review Lengths')

# Review length hist (remove the 10 shortest/longest reviews)
interval = 20
all_review_len_sorted_clip = sorted(all_review_len)[10:-10]
plt.hist(all_review_len_sorted_clip, bins=[v*interval for v in list(
    np.array(range(np.max(all_review_len_sorted_clip)//interval)))],
    color='#990000')
plt.xlabel('Words')
plt.ylabel('# of Reviews')
plt.show()
fig.savefig('asset/review_len_hist.png')

In [ ]:
unique_ratings = np.unique(all_rating)
bins = [v*interval for v in list(np.array(range(np.max(all_review_len)//interval)))]
gs = gridspec.GridSpec(4, 1) 
fig = plt.figure(figsize=(18, 12))
for i, unique_rating in enumerate(unique_ratings):
    plt.subplot(gs[i])
    # Plot histograms of ratings
    
    # Review length hist
    all_review_len_rating = []
    for j, r in enumerate(all_rating):
        if r == unique_rating:
            all_review_len_rating.append(all_review_len[j])
            
    plt.title('Histogram of Review Lengths with Rating {} (the average length: {:.2f})'.format(
        unique_rating, np.mean(all_review_len_rating)))
    
    plt.hist(all_review_len_rating, bins=bins, color='#990000')
    if i == len(unique_ratings)-1:
        plt.xlabel('Words')
    else:
        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            labelbottom=False) # labels along the bottom edge are off    
    plt.ylim(0, 180)
    plt.xlim(0, 1500)     
    plt.ylabel('# of reviews')      
plt.show()    
fig.savefig('asset/review_len_hist_rating.png')

In [ ]:
# high variance (both 1 and 8 but not 3 or 6)
count = 0
for m in meta_list:
    if 1 in m.rating and 8 in m.rating and 6 not in m.rating and 3 not in m.rating:
        count += 1
        print('[{}] {} {} {}'.format(count, m.title, m.url, m.rating))

In [ ]:
# Write all the data to README
import datetime
with open('README.md', 'r+') as readme:
    lines = readme.readlines()

data_title = '## <a id="Data"></a>All ICLR 2020 OpenReview data\n'
idx = lines.index(data_title)
lines = lines[:idx]

with open('README.md', 'w') as readme:
    for line in lines:
        readme.write(line)
    readme.write(data_title)
    readme.write('Collected at {}\n\n'.format(datetime.datetime.now()))
    readme.write('Number of submissions: {} (withdrawn/desk reject submissions: {})\n\n'.format(
        len(meta_list), num_withdrawn))
    readme.write('| Rank | Average Rating | Title | Ratings | Variance | Decision |\n')
    readme.write('| --- | --- | --- | --- | --- | --- |\n')
    non_empty_rating_meta_list = [m for m in meta_list if not len(m.rating)==0]
    empty_rating_meta_list = [m for m in meta_list if len(m.rating)==0]    
    sorted_idx = np.argsort([np.mean(m.rating) for m in non_empty_rating_meta_list])[::-1]
    for i, idx in enumerate(sorted_idx):
        m = non_empty_rating_meta_list[idx]
        readme.write('| {} | {:.2f} | [{}]({}) | {} | {:.2f} | {} |\n'.format(
            i+1, np.mean(m.rating), 
            m.title if not (m.withdrawn or m.desk_reject) else '~~'+m.title+'~~',  
            m.url, ', '.join([str(r) for r in list(m.rating)]),
            np.std(m.rating), m.decision
        ))
    for i, m in enumerate(empty_rating_meta_list):
        readme.write('| {} | {:.2f} | [{}]({}) | {} | {:.2f} | {} |\n'.format(
            i+1+len(non_empty_rating_meta_list), np.mean(m.rating), 
            m.title if not (m.withdrawn or m.desk_reject) else '~~'+m.title+'~~',
            m.url, ', '.join([str(r) for r in list(m.rating)]),
            np.std(m.rating), m.decision
        ))

In [ ]:
if AFTER_DECISION:
    rating_mean_accept = []
    rating_mean_reject = []
    for m in meta_list:
        if len(m.rating) > 0:
            if 'Accept' in m.decision:
                rating_mean_accept.append(m.average_rating)
            else:
                rating_mean_reject.append(m.average_rating)
    print('Average rating of accepted papers: {}'.format(np.mean(rating_mean_accept)))
    print('Average rating of rejected papers: {}'.format(np.mean(rating_mean_reject)))

In [ ]:
if AFTER_DECISION:
    fig = plt.figure(figsize=(12, 7))
    plt.title('Histograms of ICLR 2020 Submission Ratings', fontsize=15)

    # accepted
    counterlist_all = sorted(Counter(rating_mean).most_common())
    counterlist_accept =  sorted(Counter(rating_mean_accept).most_common())
    for c in counterlist_all:
        if c[0] not in [ca[0] for ca in counterlist_accept]:
            counterlist_accept.append((c[0], 0))
    counterlist_accept = sorted(counterlist_accept)
    frequencies = [k[1] for k in counterlist_accept]
    freq_series = pd.Series.from_array(frequencies)
    bins = [k[0] for k in counterlist_all]

    x_labels = ['{:.2f}'.format(b) for b in bins]

    ax1 = freq_series.plot(kind='bar', color='#990000', alpha=0.5, width=0.9)

    rects = ax1.patches
    labels = [int(frequencies[i]) for i in range(len(rects))]

    for rect, label in zip(rects, labels):
        height = rect.get_height()
        if label > 0:
            ax1.text(rect.get_x() + rect.get_width() / 2, height + 2, label,
                     ha='center', va='bottom', fontsize=9, color='#990000')

    # rejected
    counterlist_reject =  sorted(Counter(rating_mean_reject).most_common())
    for c in counterlist_all:
        if c[0] not in [ca[0] for ca in counterlist_reject]:
            counterlist_reject.append((c[0], 0))
    counterlist_reject = sorted(counterlist_reject)
    frequencies = [k[1] for k in counterlist_reject]
    freq_series = pd.Series.from_array(frequencies)

    ax2 = freq_series.plot(kind='bar', color='#000099', alpha=0.5, width=0.9)
    ax2.set_xlabel('Rating per paper hist', fontsize=15)
    ax2.set_ylabel('Number of papers', fontsize=15)
    ax2.set_xticklabels(x_labels, fontsize=9)

    rects = ax2.patches[int(len(rects)/2):]
    labels = [int(frequencies[i]) for i in range(len(rects))]

    for rect, label in zip(rects, labels):
        height = rect.get_height()
        if label > 0:
            ax2.text(rect.get_x() + rect.get_width() / 2, height + 2, label,
                     ha='center', va='bottom', fontsize=9, color='#000099')

    ax1.legend(('Accepted Papers', 'Rejected Papers'), fontsize=15)
    plt.show()
    fig.savefig('asset/decision.png')

In [ ]:
# Compare data crawled at different time
from collections import Counter
data1 = 'data_20191105041300.hdf5'
data2 = 'data_20191115155104.hdf5'

def get_rating(filename):
    print('Loading {} ...'.format(filename))
    meta_list = crawl_meta(meta_hdf5=filename)    
    rating = []    
    rating_mean = []
    for m in meta_list:
        rating.extend(m.rating)
        if len(m.rating) > 0:
            rating_mean.append(m.average_rating)
    print('Average rating: {:.4f}'.format(np.mean(rating)))    
    
    crawl_time = time.strftime(
        '%m/%d/%Y %I:%M:%S %p', 
        time.strptime(filename.split('_')[-1].split('.')[0], '%Y%m%d%H%M%S'))
    return rating_mean, rating, crawl_time

rating_mean_1, rating_1, time1 = get_rating(data1)
rating_mean_2, rating_2, time2 = get_rating(data2)

In [ ]:
def distribution_differet_plot(
        data1, data2, name, xlabel, ylabel, figname=None,
        legend_height_distance=0.3, legend_height_offset=1,
    ):
    fig = plt.figure(figsize=(15, 8))
    plt.title(name, fontsize=15)


    # data1
    counterlist_all = sorted(Counter(data1['data']+data2['data']).most_common())
    counterlist2 =  sorted(Counter(data1['data']).most_common())
    for c in counterlist_all:
        if c[0] not in [ca[0] for ca in counterlist2]:
            counterlist2.append((c[0], 0))
    counterlist2 = sorted(counterlist2)
    frequencies = [k[1] for k in counterlist2]
    frequencies_normalized = [float(f) / np.sum(frequencies) * 100 for f in frequencies]
    freq_series = pd.Series.from_array(frequencies_normalized)
    bins = [k[0] for k in counterlist_all]

    x_labels = ['{:.2f}'.format(b) for b in bins]

    ax1 = freq_series.plot(kind='bar', color='#990000', alpha=0.25, width=0.9)

    # rects = ax1.patches
    rects1 = ax1.patches # [:int(len(rects)/2)]    
    labels1 = ['{:.1f}'.format(frequencies_normalized[i]) for i in range(len(rects1))]    

    # data2
    counterlist2 =  sorted(Counter(data2['data']).most_common())
    for c in counterlist_all:
        if c[0] not in [ca[0] for ca in counterlist2]:
            counterlist2.append((c[0], 0))
    counterlist2 = sorted(counterlist2)
    frequencies = [k[1] for k in counterlist2]
    frequencies_normalized = [float(f) / np.sum(frequencies) * 100 for f in frequencies]
    freq_series = pd.Series.from_array(frequencies_normalized)

    ax2 = freq_series.plot(kind='bar', color='#000099', alpha=0.25, width=0.9)
    ax2.set_xlabel(xlabel, fontsize=15)
    ax2.set_ylabel(ylabel, fontsize=15)
    ax2.set_xticklabels(x_labels, fontsize=9)

    rects2 = ax2.patches[int(len(rects1)/2):]
    labels2 = ['{:.1f}'.format(frequencies_normalized[i]) for i in range(len(rects2))]

    # import ipdb; ipdb.set_trace()
    for rect1, rect2, label1, label2 in zip(rects1, rects2, labels1, labels2):
        height = max(rect1.get_height(), rect2.get_height())
        if rect1.get_height() > rect2.get_height():
            legend_height_offse_sign = 1
        else:
            legend_height_offse_sign = -1
        if float(label1) > 0:
            ax1.text(rect1.get_x() + rect1.get_width() / 2, 
                     height + legend_height_offset + legend_height_offse_sign*legend_height_distance, 
                     label1+'%', ha='center', va='bottom', fontsize=9, color='#990000')
        if float(label2) > 0:
            ax2.text(rect2.get_x() + rect2.get_width() / 2, 
                     height + legend_height_offset - legend_height_offse_sign*legend_height_distance, 
                     label2+'%', ha='center', va='bottom', fontsize=9, color='#000099')
                
    ax1.legend((data1['legend'], data2['legend']), fontsize=15)
    plt.show()
    if figname is not None:
        fig.savefig(figname)

In [ ]:
distribution_differet_plot(
    {'data': rating_1, 'legend': time1},
    {'data': rating_2, 'legend': time2},
    'ICLR 2020 Submission Rating Change During Rebuttal',
    'Rating',
    'Percentage (%)',
    'asset/rating_difference.png',
    legend_height_distance=0.5,
    legend_height_offset=0.4,    
)        

distribution_differet_plot(
    {'data': rating_mean_1, 'legend': time1},
    {'data': rating_mean_2, 'legend': time2},
    'ICLR 2020 Submission Average Rating Change During Rebuttal',
    'Average Rating',
    'Percentage (%)',
    'asset/rating_mean_difference.png',
    legend_height_distance=0.2,
    legend_height_offset=0.2,    
)

In [ ]:
# write a linux script that can downloads all the paper pdf with its title
with open('download_all_paper.sh', 'w') as f:
    for m in meta_list:
        data_id = m.url.split('=')[-1]
        f.write('curl -o {}.pdf https://openreview.net/pdf?id={}\n'.format(m.title.replace(' ', '_'), data_id))